# Module 5: Assignment - Comparison Exercise - Analyzing Deep Learning Frameworks

In [ ]:
%pip install kaggle

In [ ]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()

# Download the bird identification dataset from Kaggle https://www.kaggle.com/datasets/gpiosenka/100-bird-species
dataset_name = 'gpiosenka/100-bird-species'
path_to_download = 'birds'  # Specify your download path here

api.dataset_download_files(dataset_name, path=path_to_download, unzip=True)

print(f'Dataset downloaded and extracted to: {path_to_download}')


### Training with the Kaggle 525 Bird Species dataset using TensorFlow

In [11]:
# Import Tensor Flow
import tensorflow
# Import Keras
from tensorflow import keras
# Import Numpy
import numpy as np
# Import Matplotlib

# import the dataset
train_data = keras.utils.image_dataset_from_directory('birds/train', image_size=(224, 224), batch_size=32)
test_data = keras.utils.image_dataset_from_directory('birds/test', image_size=(224, 224), batch_size=32)
validation_data = keras.utils.image_dataset_from_directory('birds/valid', image_size=(224, 224), batch_size=32)
# Load the class names from csv file
class_names = np.loadtxt('birds/birds.csv', delimiter=',', usecols=(1,), dtype=str, skiprows=1)
# Print the class names
print(class_names)
# Print the shape of the training data
print(train_data)
# Print the shape of the test data
print(test_data)
# Print the shape of the validation data
print(validation_data)


# Example of normalization for image data
train_data = train_data.map(lambda x, y: (x / 255.0, y))
test_data = test_data.map(lambda x, y: (x / 255.0, y))
validation_data = validation_data.map(lambda x, y: (x / 255.0, y))

# Create a CNN model
tf_model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(len(class_names), activation='softmax')
])
# Compile the model
tf_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Fit the model
tf_model.fit(train_data, epochs=5, validation_data=validation_data)

# Evaluate the model
tf_model.evaluate(test_data)

# Make predictions
predictions = tf_model.predict(test_data)
# Print the predictionsf
print(predictions)


Found 84635 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.
['train/ABBOTTS BABBLER/001.jpg' 'train/ABBOTTS BABBLER/007.jpg'
 'train/ABBOTTS BABBLER/008.jpg' ... 'valid/BLACK BREASTED PUFFBIRD/1.jpg'
 'valid/BLACK BREASTED PUFFBIRD/2.jpg'
 'valid/BLACK BREASTED PUFFBIRD/5.jpg']
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Epoch 1/5
2645/2645 [==============================] - 357s 135ms/step - loss: 5.1148 - accuracy: 0.0615 - val_loss: 4.2214 - val_accuracy: 0.1250
Epoch 2/5
2645/2645 [===========================

### Training with the 525 Bird Species dataset using PyTorch

In [12]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import tqdm


# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using:", device)


# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Load datasets
train_dataset = datasets.ImageFolder('birds/train', transform=transform)
test_dataset = datasets.ImageFolder('birds/test', transform=transform)
validation_dataset = datasets.ImageFolder('birds/valid', transform=transform)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Load the class names from csv file
class_names = np.loadtxt('birds/birds.csv', delimiter=',', usecols=(1,), dtype=str, skiprows=1)
# Print the class names
print(class_names)
# Print the shape of the training data
print(train_dataset)
# Print the shape of the test data
print(test_dataset)
# Print the shape of the validation data
print(validation_dataset)


# Creating a temporary model to determine input size for first linear layer
temp_model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, 3, 1),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten()
)

x, _ = next(iter(train_loader))
x = temp_model(x)
print('Temporary Model Shape',x.shape)


# Create a CNN model with PyTorch
model = torch.nn.Sequential(
    torch.nn.Conv2d(3, 32, 3, 1),
    torch.nn.MaxPool2d(2),
    torch.nn.Flatten(),
    torch.nn.Linear(394272, 128),
    torch.nn.Linear(128, len(class_names))
)

# move the model to the GPU
model.to(device)


# Define the loss function
loss_fn = torch.nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters())

# Train the model
for epoch in range(5):
    progress_bar = tqdm.tqdm(enumerate(train_loader), total=len(train_loader))
    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progress_bar.set_description(f'Epoch {epoch + 1}/{5}, Loss: {loss.item():.4f}')

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    progress_bar = tqdm.tqdm(enumerate(test_loader), total=len(test_loader))
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        _, predicted = torch.max(y_pred, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
        progress_bar.set_description(f'Accuracy: {correct / total}')

print(f'Accuracy: {correct / total}')


Using: cuda
['train/ABBOTTS BABBLER/001.jpg' 'train/ABBOTTS BABBLER/007.jpg'
 'train/ABBOTTS BABBLER/008.jpg' ... 'valid/BLACK BREASTED PUFFBIRD/1.jpg'
 'valid/BLACK BREASTED PUFFBIRD/2.jpg'
 'valid/BLACK BREASTED PUFFBIRD/5.jpg']
Dataset ImageFolder
    Number of datapoints: 84635
    Root location: birds/train
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 2625
    Root location: birds/test
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )
Dataset ImageFolder
    Number of datapoints: 2625
    Root location: birds/valid
    StandardTransform
Transform: Compose(
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           

Epoch 3/10, Loss: 2.0150:   0%|          | 0/2645 [1:35:23<?, ?it/s]


In [ ]:
import tqdm

# Train the model with tqdm
for epoch in range(10):
    progress_bar = tqdm.tqdm(train_loader, desc=f'Epoch {epoch+1}/{10}', leave=False)
    for x, y in progress_bar:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        progress_bar.set_postfix({'Loss': loss.item()})


In [7]:
%pip install psutil gputil matplotlib


  Preparing metadata (setup.py) ... done
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7411 sha256=41580714a61bed5e1d5c8bc2559d4782f983f8ea05a7ef998a7ab8f3fc607098
  Stored in directory: /root/.cache/pip/wheels/ba/03/bb/7a97840eb54479b328672e15a536e49dc60da200fb21564d53
Successfully built gputil

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Import Tensor Flow
import tensorflow
# Import Keras
from tensorflow import keras
# Import Numpy
import numpy as np
# Import PSUtil
import tensorboard
# Import GPUtil
import GPUtil
# Import Matplotlib
import matplotlib.pyplot as plt
# Import Time
import time

# Import the dataset
print('Loading the dataset...')
train_data = keras.utils.image_dataset_from_directory('birds/train', image_size=(224, 224), batch_size=32)
test_data = keras.utils.image_dataset_from_directory('birds/test', image_size=(224, 224), batch_size=32)
validation_data = keras.utils.image_dataset_from_directory('birds/valid', image_size=(224, 224), batch_size=32)
print('Dataset loaded')
# Load the class names from csv file
class_names = np.loadtxt('birds/birds.csv', delimiter=',', usecols=(1,), dtype=str, skiprows=1)
# Print the class names
print(class_names)
# Print the shape of the training data
print(train_data)
# Print the shape of the test data
print(test_data)
# Print the shape of the validation data
print(validation_data)



# Example of normalization for image data
print('Normalizing the dataset...')
train_data = train_data.map(lambda x, y: (x / 255.0, y))
test_data = test_data.map(lambda x, y: (x / 255.0, y))
validation_data = validation_data.map(lambda x, y: (x / 255.0, y))
print('Dataset normalized')


# Create a CNN model
print('Creating the model...')
tf_model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(525, activation='softmax')
])
print('Model created')

# Compile the model
print('Compiling the model...')
tf_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print('Model compiled')

# Define the logs directory
logs = "logs/profile"

# Define the TensorBoard callback
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logs, histogram_freq=1, profile_batch='500,520')

# Fit the model
print('Fitting the model...')
tf_model.fit(train_data, epochs=5, validation_data=validation_data)
print('Model fit')

# Evaluate the model
print('Evaluating the model...')
tf_model.evaluate(test_data)
print('Model evaluated')

# Make predictions
predictions = tf_model.predict(test_data)
# Print the predictionsf
print(predictions)



Found 84635 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.
Found 2625 files belonging to 525 classes.
['train/ABBOTTS BABBLER/001.jpg' 'train/ABBOTTS BABBLER/007.jpg'
 'train/ABBOTTS BABBLER/008.jpg' ... 'valid/BLACK BREASTED PUFFBIRD/1.jpg'
 'valid/BLACK BREASTED PUFFBIRD/2.jpg'
 'valid/BLACK BREASTED PUFFBIRD/5.jpg']
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
Epoch 1/5


2024-02-24 23:16:48.446910: I tensorflow/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-02-24 23:16:48.446942: I tensorflow/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-02-24 23:16:48.446999: I tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1671] Profiler found 1 GPUs
2024-02-24 23:16:48.461992: E tensorflow/compiler/xla/backends/profiler/gpu/cupti_error_manager.cc:194] cuptiSubscribe: error 15: CUPTI_ERROR_NOT_INITIALIZED
2024-02-24 23:16:48.462037: E tensorflow/compiler/xla/backends/profiler/gpu/cupti_error_manager.cc:459] cuptiGetResultString: ignored due to a previous error.
2024-02-24 23:16:48.462042: E tensorflow/compiler/xla/backends/profiler/gpu/cupti_tracer.cc:1723] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error 
2024-02-24 23:16:48.462176: I tensorflow/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-02-24 23:16:48.46

1573/2645 [================>.............] - ETA: 7:24 - loss: 6.3449 - accuracy: 0.0030

KeyboardInterrupt: 